# 1. Funktionen definieren

In [ ]:
#################################Funktion um Assets einzulesen#################################
def einlesen_asset(assetsymbols, start, end, field, periode, apikey):
    try:
        import eikon as ek
        import pandas as pd
        
        ek.set_app_key(apikey)
        df = ek.get_timeseries(assetsymbols, start_date = start, end_date = end, fields = field, interval = periode)
        
        df_2,e = ek.get_data(assetsymbols, "TR.CommonName")
        df = df.set_axis(df_2.iloc[: , 1].values, axis=1, inplace=False)
        print("Download war erfolgreich!")
        return df

    except ModuleNotFoundError as m:
        print(str(m)+"."+" Bitte installiere das Modul mit 'pip install'.") 
        
    except ValueError:
        print("Das Start oder Enddatum ist falsch.")
        
    except AttributeError:
        print("Startdatum ist grösser als das Enddatum.")
        
    except NameError:
        print("Der Input ist kein String.")
        
    except TypeError:
        print("Falsche Zahl an Argumenten.")

In [ ]:
#################################Funktion um Makrodaten einzulesen#################################
def einlesen_macro(macrosymbols_refinitiv,macrosymbols_fred, start, end, apikey):
    import eikon as ek
    import pandas as pd
    import math
    ek.set_app_key(apikey)

    without_gdp = macrosymbols_refinitiv[macrosymbols_refinitiv.index != 'GDP']
    with_gdp = macrosymbols_refinitiv[macrosymbols_refinitiv.index == 'GDP']

    df_1 = ek.get_timeseries(list(without_gdp.values), start_date = start, end_date = end, interval = 'monthly')
    df_2 = ek.get_timeseries(list(with_gdp.values), start_date = start, end_date = end, interval = 'quarterly')
    df_3 = web.DataReader(list(macrosymbols_fred.values), 'fred', start, end)
    df_3.index = df_3.index + pd.Timedelta(days=-1)

    df = df_1.join(df_2, how = 'left')
    df.columns.values[len(df.columns)-1] = with_gdp.values[0]
    df = df.join(df_3, how = 'left')
    
    num = format(j+1, '.0f')
    print("Download " + num + " von 8 war Erfolgreich!")
    return df

In [ ]:
#################################Funktion die NA's zählt und anzeigt ob über 1#################################
def na_ckeck(timeseries):
    try:
        import pandas as pd
        import numpy as np
        
        anzahl = 0

        for i in range(0,len(timeseries.columns)):
            number = sum(timeseries.iloc[:, i].isna())
            if number > 0:
                anzahl += 1
                print(timeseries.columns[i] + " hat " + str(number) + " NA's ")
                
        if anzahl != 0 and anzahl != len(timeseries.columns):
            print("")
            print("Der Rest hat keine NA's.")
            
        if anzahl == 0:
            print("Der gesamte Datensatz hat keine NA's.")       
        
    except ModuleNotFoundError as m:
        print(str(m)+"."+" Bitte installiere das Modul mit 'pip install'.")  

In [ ]:
#################################Funktion die Trends anzeigt#################################
def visual_stationarity(timeseries, size):
    
    import pandas as pd
    import matplotlib.pylab as plt
    %matplotlib inline
    
    for i in range(0,len(timeseries.columns)):
        x = timeseries.iloc[: , i]
        x = np.reshape(x, len(x))
        rol_mean = x.rolling(window=size).mean() #SMA = Simple Moving Average
        rol_weighted_mean = x.ewm(span=size).mean() #EMA = Exponential Moving Average

        plt.plot(x, color = 'black',label = 'Orginal')
        plt.plot(rol_mean, color = 'red', label = 'Einfacher Moving Average')
        plt.plot(rol_weighted_mean, color = 'green', label = 'Exponentieller Moving Average')
        plt.legend(loc = 'best')
        plt.title(x.name)
        plt.show()

In [ ]:
#################################Funktion die Trends anzeigt#################################
def visual_describe(timeseries):
    
    import pandas as pd
    import matplotlib.pylab as plt
    %matplotlib inline

    for i in range(0,len(timeseries.columns)):
        mean = timeseries.iloc[:,i].groupby(pd.Grouper(freq='1Y')).mean()
        mean = np.reshape(mean, len(mean))
        median = timeseries.iloc[:,i].groupby(pd.Grouper(freq='1Y')).median()
        median = np.reshape(median, len(median))
        std = timeseries.iloc[:,i].groupby(pd.Grouper(freq='1Y')).std()
        std = np.reshape(std, len(std))

        plt.plot(mean, color = 'black',label = 'Mean')
        plt.plot(median, color = 'red', label = 'Median')
        plt.plot(std, color = 'green', label = 'Standardabweichung')
        plt.legend(loc = 'best')
        plt.title(timeseries.columns[i])
        plt.show()

In [ ]:
def dickey_fuller(timeseries):
    from statsmodels.tsa.stattools import adfuller
    
    anzahl = 0
    
    for i in range(0,len(timeseries.columns)):
        result = adfuller(timeseries.iloc[:, i])
        if result[1] >= 0.05:
            anzahl += 1
            print("Der p-Value von " + str(timeseries.columns[i]) + " ist %f" % result[1] + ". Somit ist die Zeitreihe nicht stationär.")
            
    if anzahl == 0:
        print('Der gesamte Datensatz ist Stationär.')
        
    if anzahl == len(timeseries.columns):
        print("")
        print('Somit ist der gesamte Datensatz nicht Stationär.')     
        
    if anzahl != 0 and anzahl != len(timeseries.columns):
        print("")
        print("Der Rest ist Stationär.")

In [ ]:
#################################Funktion die Normalverteilung berechnet (Jarque Bera)#################################
def jarque_bera(timeseries):
    import scipy.stats as stats
    
    anzahl = 0
    
    for i in range(0,len(timeseries.columns)):
        result = stats.jarque_bera(timeseries.iloc[: , i])
        
        if result[1] <= 0.05:
            anzahl += 1
            print("Der p-Value von " + str(timeseries.columns[i]) + " ist %f" % result[1] + ". Somit ist die Zeitreihe nicht normalverteilt.")
            
    if anzahl == 0:
        print('Der gesamte Datensatz ist Normalverteeilt.')
        
    if anzahl == len(timeseries.columns):
        print("")
        print('Somit ist der gesamte Datensatz nicht Normalverteilt.')     
        
    if anzahl != 0 and anzahl != len(timeseries.columns):
        print("")
        print("Der Rest ist Normalverteilt.")        

In [ ]:
#################################Funktion die Normalverteilung berechnet (Jarque Bera)#################################
def jarque_bera_res(residuals):
    import scipy.stats as stats
    
    anzahl = 0
    
    for i in range(0,len(residuals.columns)):
        result = stats.jarque_bera(residuals.iloc[: , i])
        
        if result[1] <= 0.05:
            anzahl += 1
            print("Der p-Value von " + str(residuals.columns[i]) + " ist %f" % result[1] + ". Somit sind die Residuen nicht normalverteilt.")
            
    if anzahl == 0:
        print('Alle Residuen sind Normalverteeilt.')
        
    if anzahl == len(residuals.columns):
        print("")
        print('Somit sind alle Residuen nicht Normalverteilt.')     
        
    if anzahl != 0 and anzahl != len(residuals.columns):
        print("")
        print("Die restliche Residuen sind Normalverteilt.")  

In [ ]:
#################################Funktion die Normalverteilung berechnet (Sharpiro Wilk)#################################
def shapiro_wilk(timeseries):
    import scipy as sp
    
    anzahl = 0
    
    for i in range(0,len(timeseries.columns)):
        result = sp.stats.shapiro(timeseries.iloc[: , i])
        
        if result[1] <= 0.05:
            anzahl += 1
            print("Der p-Value von " + str(timeseries.columns[i]) + " ist %f" % result[1] + ". Somit ist die Zeitreihe nicht normalverteilt.")
            
    if anzahl == 0:
        print('Der gesamte Datensatz ist Normalverteeilt.')
        
    if anzahl == len(timeseries.columns):
        print("")
        print('Somit ist der gesamte Datensatz nicht Normalverteilt.')     
        
    if anzahl != 0 and anzahl != len(timeseries.columns):
        print("")
        print("Der Rest ist Normalverteilt.")             

In [ ]:
#################################Funktion die eine Acf zeichnet#################################
def acf_plot(timeseries, lag):
    from statsmodels.tsa.stattools import acf, pacf
    from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
    import numpy as np
    
    for i in range(0,len(timeseries.columns)):
        x = np.log(timeseries.iloc[: , i]) - np.log(timeseries.iloc[: , i].shift(periods = 1))
        y = x.dropna()
        plot_acf(y, lags=lag)
        plot_pacf(y, lags=lag)

In [ ]:
#Log Transformation falls nicht Stationär
def logtransform(timeseries):
    from statsmodels.tsa.stattools import adfuller
    
    timeseries = timeseries.dropna()
  
    def findlog(x):
        import numpy as np
        if x > 0:
            log = np.log(x)
        elif x < 0:
            log = np.log(x*-1)*-1
        elif x == 0:
            log = 0
        return log
    
    for i in range(0,len(timeseries.columns)):
        result = adfuller(timeseries.iloc[:, i])

        if result[1] >= 0.05:
            timeseries.iloc[1:, i] = (timeseries.iloc[:, i].apply(findlog) - timeseries.iloc[:, i].apply(findlog).shift(1)).dropna()
    
    return timeseries

In [ ]:
#################################Funktion die Boxplots anzeigt#################################
def boxplot(timeseries):
    import pandas as pd
    import matplotlib.pyplot as plt
    import lasio

    fig, axs = plt.subplots(1, len(timeseries.columns), figsize=(20,5))

    for i, ax in enumerate(axs.flat):
        ax.boxplot(timeseries.iloc[:,i])
        ax.set_title(timeseries.columns[i])
        ax.tick_params(axis='y', labelsize=14)

    plt.tight_layout()

In [ ]:
#################################Funktion die ein Rolling Regression macht#################################

def rolling_regression(y, x, window=60):
    x = x.dropna()
    y = y.dropna()

    if x.index.size > y.index.size:
        x = x[y.index]
    else:
        y = y[x.index]

    if x.index.size < window:
        return None
    else:
        estimate_data = []
        for i in range(window, x.index.size+1):
            X_slice = x.values[i-window:i,:] # always index in np as opposed to pandas, much faster
            y_slice = y.values[i-window:i]
            coeff = np.dot(np.dot(np.linalg.inv(np.dot(X_slice.T, X_slice)), X_slice.T), y_slice)
            estimate_data.append(coeff[0] * x.values[window-1] + coeff[1])
        estimate = pd.Series(data=estimate_data, index=x.index[window-1:]) 
        return estimate

# 2. Daten einlesen

## 2.1 Assetdaten

In [ ]:
apikey = ('189c04ee23da4cb684ea4e555603680fdda96a95')
assetsymbols = ["AAPL.O", "NVDA.O", "GOOGL.O","CSCO.O", "ASML.O", "SAPG.DE", "LOGN.S", "0700.HK", "005930.KS" ,"6758.T"]

assetdata = einlesen_asset(assetsymbols,"2012-03-20", "2022-03-20", "CLOSE", "monthly", apikey)

## 2.2 Macrodaten

In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web

macrosymbols_refinitiv = {'USA': ["USGDPF=ECI","USUNR=ECI","USCPI=ECI",np.nan,"USRSL=ECI","USIP=ECI","USIMP=ECI","USEXP=ECI"]
        ,'Niederlande': ["NLGDP=ECI","NLUNRS=ECI","NLCPIY=ECI",np.nan,"NLRSLY=ECI",np.nan,np.nan,np.nan]
        ,"Europa": ['EUGDP=ECI',"EUUNR=ECI",np.nan,"EUPPI=ECI",np.nan,"EUIP=ECI",np.nan,np.nan]
        ,"Schweiz": ["CHGDP=ECI","CHJOB=ECI","CHCPI=ECI",np.nan,"CHRS=ECI",np.nan,np.nan,np.nan]
        ,"Deutschland": ["DEGDP=ECI","DEUNR=ECI","DECPI=ECI","DEWPI=ECI","DERSL=ECI","DEIP=ECI","DEIMY=ECI","DEEXPY=ECI"]
        ,"China": ["CNGDPA=ECI",np.nan,"CNCPI=ECI",np.nan,"CNRSL=ECI","CNIO=ECI","CNIMP=ECI","CNEXP=ECI"]
        ,"Japan": ["JPGD1=ECI","JPUNR=ECI","JPCPI=ECI",np.nan,"JPRSLS=ECI","JPIP1=ECI","JPIMPY=ECI","JPEXPY=ECI"]
        ,"Südkorea": ["KRGDQA=ECI","KRUNR=ECI","KRCPI=ECI",np.nan,np.nan,"KRIO=ECI","KRIMP=ECI","KREXP=ECI"]
        }

macrosymbols_refinitiv = pd.DataFrame(data=macrosymbols_refinitiv, index = ["GDP","Unemployment Rate","CPI Index","Wholesale Price Index","Retailsales","Industrial Output","Import","Export"])
macrosymbols_refinitiv

In [ ]:
macrosymbols_fred = {'USA': [np.nan,'INDPRO','PCUAWHLTRAWHLTR',np.nan,np.nan,np.nan,np.nan]
        ,'Niederlande': [np.nan,'NLDPROINDMISMEI',np.nan,np.nan,np.nan,'XTIMVA01NLM667S','XTEXVA01NLM664S']
        ,"Europa": ['CPHPTT01EZM659N','EA19PRINTO01GPSAM',np.nan,'EA19SLRTTO02IXOBSAM',np.nan,'XTIMVA01EZM667S','XTEXVA01EZM667S']
        ,"Schweiz": [np.nan,'CHEPROINDQISMEI',np.nan,np.nan,np.nan,'XTIMVA01CHM664S','XTEXVA01CHM667S']
        ,"Deutschland": [np.nan,'DEUPROINDMISMEI',np.nan,np.nan,np.nan,np.nan,np.nan]
        ,"China": [np.nan,'CHNPRINTO01IXPYM',np.nan,np.nan,np.nan,np.nan,np.nan]
        ,"Japan": [np.nan,'JPNPROINDMISMEI',np.nan,np.nan,np.nan,np.nan,np.nan]
        ,"Südkorea": [np.nan,'KORPROINDMISMEI',np.nan,'KORSARTMISMEI',np.nan,np.nan,np.nan]
        }
macrosymbols_fred = pd.DataFrame(data=macrosymbols_fred, index = ["CPI Index","Produktionsauslastung","Wholesale Price Index","Retailsales","Industrial Output","Import","Export"])
macrosymbols_fred

In [ ]:
apikey = ('189c04ee23da4cb684ea4e555603680fdda96a95')

j = 0
for i in ['us', 'nl', 'eu','ch','de','cn','jp','sk']:
    exec(f'macrodata_{i} = einlesen_macro(macrosymbols_refinitiv.iloc[:,j].dropna(),macrosymbols_fred.iloc[:,j].dropna(),"2012-03-20", "2022-03-20", apikey)')
    j += 1
    
macrodata = macrodata_us.join(macrodata_nl, how='left').join(macrodata_eu, how='left').join(macrodata_ch, how='left').join(macrodata_de, how='left').join(macrodata_cn, how='left').join(macrodata_jp, how='left').join(macrodata_sk, how='left')    

# 3. Assetdaten analysieren

## 3.1 NA's prüfen

In [ ]:
na_ckeck(assetdata)

## 3.2 Übersicht mittels Liniendiagramm

In [ ]:
import chart_studio
import cufflinks as cf

cf.set_config_file(offline = True)

assetdata.normalize().iplot()

## 3.3 Boxplot (Orginal)

In [ ]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt

for i in range(0,len(assetdata.columns)):
    fig, ax = plt.subplots(figsize=(12,5))
    seaborn.boxplot(x=assetdata.iloc[:,i].index.year, y= assetdata.iloc[:,i].values, data=assetdata, ax=ax)
    ax.set_title(assetdata.columns[i])

## 3.4 Deskriptive Statistik (Orginal)

In [ ]:
visual_describe(assetdata)

## 3.5 Stationarität (Orginal)

In [ ]:
#Optisch
visual_stationarity(assetdata, 24)

In [ ]:
#Mathematisch
dickey_fuller(assetdata)

## 3.6 Normalverteilung (Orginal)

In [ ]:
#Optisch
import seaborn as sns
import matplotlib.pylab as plt

for i in range(0,len(assetdata.columns)):
    y = assetdata.iloc[: , i]    
    sns.distplot(y)
    plt.title(y.name)
    plt.show()

In [ ]:
#Mathematisch (Jarque Bera)
jarque_bera(assetdata)

In [ ]:
#Mathematisch (Shapiro Wilk)
shapiro_wilk(assetdata)

## 3.7 Kompisition (Orginal)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

for i in range(0,len(assetdata.columns)):
    decompose = seasonal_decompose(assetdata.iloc[:,i],model='additive', period=12)
    decompose.plot()
    plt.show()

## 3.8 Log-Returns machen

In [ ]:
import numpy as np
assetdata_returns = (np.log(assetdata) - np.log(assetdata.shift(1))).dropna()

## 3.9 Deskriptive Statistik (Returns)

In [ ]:
visual_describe(assetdata_returns)

## 3.10 Stationarität (Returns)

In [ ]:
#Optisch
visual_stationarity(assetdata_returns, 24)

In [ ]:
#Mathematisch
dickey_fuller(assetdata_returns)

## 3.11 Normalverteilung (Returns)

In [ ]:
#Optisch

import seaborn as sns

import matplotlib.pylab as plt
for i in range(0,len(assetdata_returns.columns)):
    y = assetdata_returns.iloc[: , i]    
    sns.distplot(y)
    plt.title(y.name)
    plt.show()

In [ ]:
#Mathematsch (Shapiro Wilk)
shapiro_wilk(assetdata_returns)

## 3.12 Boxplot (Returns)

In [ ]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt

for i in range(0,len(assetdata_returns.columns)):
    fig, ax = plt.subplots(figsize=(12,5))
    seaborn.boxplot(x=assetdata_returns.iloc[:,i].index.year, y= assetdata_returns.iloc[:,i].values, data=assetdata, ax=ax)
    ax.set_title(assetdata_returns.columns[i])

## 3.13 Kompositionen (Returns)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

for i in range(0,len(assetdata_returns.columns)):
    decompose = seasonal_decompose(assetdata_returns.iloc[:,i],model='additive', period=12)
    decompose.plot()
    plt.show()

# 4. Makrodaten analysieren

## 4.1 NA's prüfen

In [ ]:
na_ckeck(macrodata)

In [ ]:
macrodata_us = macrodata_us.fillna(method='ffill')
macrodata_nl = macrodata_nl.fillna(method='ffill')
macrodata_eu = macrodata_eu.fillna(method='ffill')
macrodata_ch = macrodata_ch.fillna(method='ffill')
macrodata_de = macrodata_de.fillna(method='ffill')
macrodata_cn = macrodata_cn.fillna(method='ffill')
macrodata_jp = macrodata_jp.fillna(method='ffill')
macrodata_sk = macrodata_sk.fillna(method='ffill')
macrodata = macrodata_us.join(macrodata_nl, how='left').join(macrodata_eu, how='left').join(macrodata_ch, how='left').join(macrodata_de, how='left').join(macrodata_cn, how='left').join(macrodata_jp, how='left').join(macrodata_sk, how='left')    

In [ ]:
na_ckeck(macrodata)
#KRIMP hat nicht geklappt weil es vom Start het NA's hat daher wird die 'backfill' Methode verwendet

In [ ]:
macrodata['KRIMP=ECI'] = macrodata['KRIMP=ECI'].fillna(method='backfill')

In [ ]:
na_ckeck(macrodata)

## 4.2 Multikollinearität

In [ ]:
corr_us = macrodata_us.corr()
corr_nl = macrodata_nl.corr()
corr_eu = macrodata_eu.corr()
corr_ch = macrodata_ch.corr()
corr_de = macrodata_de.corr()
corr_cn = macrodata_cn.corr()
corr_jp = macrodata_jp.corr()
corr_sk = macrodata_sk.corr()

In [ ]:
import seaborn as sns
sns.heatmap(corr_us, xticklabels=corr_us.columns.values,yticklabels=corr_us.columns.values)

macrodata_us = macrodata_us.drop('INDPRO', 1)

In [ ]:
sns.heatmap(corr_nl, xticklabels=corr_nl.columns.values,yticklabels=corr_nl.columns.values)

macrodata_nl = macrodata_nl.drop('XTEXVA01NLM664S', 1)

In [ ]:
sns.heatmap(corr_eu, xticklabels=corr_eu.columns.values,yticklabels=corr_eu.columns.values)

macrodata_eu = macrodata_eu.drop('EA19SLRTTO02IXOBSAM', 1)

In [ ]:
sns.heatmap(corr_ch, xticklabels=corr_ch.columns.values,yticklabels=corr_ch.columns.values)

macrodata_ch = macrodata_ch.drop('XTIMVA01CHM664S', 1)

In [ ]:
sns.heatmap(corr_de, xticklabels=corr_de.columns.values,yticklabels=corr_de.columns.values)

macrodata_de = macrodata_de.drop(['DEUPROINDMISMEI','DEWPI=ECI','DEWPI=ECI'], 1)

In [ ]:
sns.heatmap(corr_cn, xticklabels=corr_cn.columns.values,yticklabels=corr_cn.columns.values)

macrodata_cn = macrodata_cn.drop(['CNIO=ECI','CNEXP=ECI','CHNPRINTO01IXPYM'], 1)

In [ ]:
sns.heatmap(corr_jp, xticklabels=corr_jp.columns.values,yticklabels=corr_jp.columns.values)

macrodata_jp = macrodata_jp.drop(['JPGD1=ECI','JPIP1=ECI','JPNPROINDMISMEI'], 1)

In [ ]:
sns.heatmap(corr_sk, xticklabels=corr_sk.columns.values,yticklabels=corr_sk.columns.values)

macrodata_sk = macrodata_sk.drop(['KRIMP=ECI','KORSARTMISMEI'], 1)

## 4.3 Boxplot

In [ ]:
boxplot(macrodata_us.dropna())

In [ ]:
boxplot(macrodata_nl.dropna())

In [ ]:
boxplot(macrodata_eu.dropna())

In [ ]:
boxplot(macrodata_ch.dropna())

In [ ]:
boxplot(macrodata_de.dropna())

In [ ]:
boxplot(macrodata_cn.dropna())

In [ ]:
boxplot(macrodata_jp.dropna())

In [ ]:
boxplot(macrodata_sk.dropna())

## 4.4 Deskriptive Statistik

In [ ]:
macrodata = macrodata_us.join(macrodata_nl, how='left').join(macrodata_eu, how='left').join(macrodata_ch, how='left').join(macrodata_de, how='left').join(macrodata_cn, how='left').join(macrodata_jp, how='left').join(macrodata_sk, how='left')    
macrodata.describe()

## 4.5 Stationarität (Orginal)

In [ ]:
dickey_fuller(macrodata)

## 4.6 Normalverteilung (Orginal)

In [ ]:
shapiro_wilk(macrodata)

## 4.7 Stationarität (Returns)

In [ ]:
macrodata_returns = logtransform(macrodata)

In [ ]:
dickey_fuller(macrodata_returns)
#KRUNR=ECI ist nicht stationär also Raus.

In [ ]:
macrodata_returns = macrodata_returns.drop('KRUNR=ECI', 1)

In [ ]:
dickey_fuller(macrodata_returns)

## 4.8 Normalverteilung (Returns)

In [ ]:
shapiro_wilk(macrodata_returns)

# 5. Modellieren

## 5.1 Zeitreihe laggen

In [ ]:
assetdata_returns["Apple Inc Lag"] = assetdata_returns['Apple Inc'].shift(1)
assetdata_returns["NVIDIA Corp Lag"] = assetdata_returns['NVIDIA Corp'].shift(1)
assetdata_returns["Alphabet Inc Lag"] = assetdata_returns['Alphabet Inc'].shift(1)
assetdata_returns["Cisco Systems Inc Lag"] = assetdata_returns['Cisco Systems Inc'].shift(1)
assetdata_returns["ASML Holding NV Lag"] = assetdata_returns['ASML Holding NV'].shift(1)
assetdata_returns["SAP SE Lag"] = assetdata_returns['SAP SE'].shift(1)
assetdata_returns["Logitech International SA Lag"] = assetdata_returns['Logitech International SA'].shift(1)
assetdata_returns["Tencent Holdings Ltd Lag"] = assetdata_returns['Tencent Holdings Ltd'].shift(1)
assetdata_returns["Samsung Electronics Co Ltd Lag"] = assetdata_returns['Samsung Electronics Co Ltd'].shift(1)
assetdata_returns["Sony Group Corp Lag"] = assetdata_returns['Sony Group Corp'].shift(1)

## 5.2 Makromatrix erstellen (erste Modelle)

In [ ]:
#Anzahl Makrodaten pro Land
country = {'US': [8], 'NL': [6], 'EU': [8], 'CH': [6], 'DE': [7], 'CN': [4], 'JP': [5], 'SK': [5]}
anz_macrodata_country = pd.DataFrame(data=country)
anz_macrodata_country

In [ ]:
#Anzahl Makrodaten pro Asset
asset = {'Apple Inc': [0,8,8], 'NVIDIA Corp': [0,8,8], 'Alphabet Inc': [0,8,8], 'Cisco Systems Inc': [0,8,8], 'ASML Holding NV': [8,14,6], 'SAP SE': [28,35,7], 'Logitech International SA': [22,28,6], 'Tencent Holdings Ltd': [35,39,4], 'Samsung Electronics Co Ltd': [39,44,5], 'Sony Group Corp': [44,49,5]}
anz_macrodata_asset = pd.DataFrame(data=asset, index = ['Index von', 'Index bis', 'Anzahl Werte'])
anz_macrodata_asset

## 5.3 Regressionsmodelle erstellen (erstes Modell)

In [ ]:
import statsmodels.api as sm

modellenamen = ['fit_apple', 'fit_nvidia', 'fit_alphabet', 'fit_cisco', 'fit_asml', 'fit_sap', 'fit_logitech', 'fit_tencent', 'fit_samsung', 'fit_sony']
    
for i in range(0,len(assetdata_returns.columns)-10):
    #Assetdaten
    y = assetdata_returns.iloc[:,i]
    x_1 = assetdata_returns.iloc[:,i+10].to_frame()
    
    #Macrodaten
    von = anz_macrodata_asset.iloc[0,i]
    bis = anz_macrodata_asset.iloc[1,i]
    x_2 = macrodata_returns.iloc[1:,von:bis]
    
    x = x_1.join(x_2, how = 'left')
    x = sm.add_constant(x)
        
    exec(f'{modellenamen[i]} = sm.OLS(y.astype(float),x.astype(float), missing = "drop").fit()')

In [ ]:
import statsmodels.api as sm

modellenamen_eu = ['fit_asml_eu', 'fit_sap_eu', 'fit_logitech_eu']
    
for i in range(3,6):
    #Assetdaten
    y = assetdata_returns.iloc[:,i]
    x_1 = assetdata_returns.iloc[:,i+10].to_frame()
    
    #Macrodaten
    x_2 = macrodata_returns.iloc[1:,16:25]
    
    x = x_1.join(x_2, how = 'left')
    x = sm.add_constant(x)
        
    exec(f'{modellenamen_eu[i-3]} = sm.OLS(y.astype(float),x.astype(float), missing = "drop").fit()')

## 5.4 Variablensignifikant prüfen (erstes Modell)

In [ ]:
modellenamen = ['fit_apple', 'fit_nvidia', 'fit_alphabet', 'fit_cisco', 'fit_asml', 'fit_sap', 'fit_logitech', 'fit_tencent', 'fit_samsung', 'fit_sony', 'fit_asml_eu', 'fit_sap_eu', 'fit_logitech_eu']

for i in range(0,len(modellenamen)):
    exec(f'print({modellenamen[i]}.summary())')

## 5.5 Variablenselektion machen (R-Studio)

In [ ]:
total = assetdata_returns.join(macrodata_returns[1:120], how = 'left')

In [ ]:
modellenamen = ['fit_apple', 'fit_nvidia', 'fit_alphabet', 'fit_cisco', 'fit_asml', 'fit_sap', 'fit_logitech', 'fit_tencent', 'fit_samsung', 'fit_sony', 'fit_asml_eu', 'fit_sap_eu', 'fit_logitech_eu']
modelle = ["Apple Inc ~ USGDPF=ECI"
           ,"NVIDIA Corp ~ USGDPF=ECI"
           ,"Alphabet Inc ~ USUNR=ECI"
           ,"Cisco Systems Inc ~ Cisco Systems Inc Lag + USRSL=ECI + USIP=ECI"
           ,"ASML Holding NV ~ XTIMVA01NLM667S"
           ,"SAP SE ~ DEGDP=ECI"
           ,"Logitech International SA ~ Logitech International SA Lag"
           ,"Tencent Holdings Ltd ~ 1"
           ,"Samsung Electronics Co Ltd ~ 1"
           ,"Sony Group Corp ~ 1"
           ,"ASML Holding NV ~ EUPPI=ECI"
           ,"SAP SE ~ EUGDP=ECI"
           ,"Logitech International SA ~ Logitech International SA"
          ]

modelle_selektiert = pd.DataFrame(data=modelle, index = modellenamen)
modelle_selektiert

## 5.6 Regressionsmodelle erstellen (selektiertes Modell)

In [ ]:
y_werte = assetdata_returns.iloc[:,[0,1,2,3,4,4,5,5,6]]
rename = ['Apple Inc', 'NVIDIA Corp', 'Alphabet Inc', 'Cisco Systems Inc',
       'ASML Holding NV', 'ASML Holding NV EU', 'SAP SE', 'SAP SE EU',
       'Logitech International SA']
y_werte.columns = rename

In [ ]:
x_werte = macrodata_returns[['USGDPF=ECI', 'USUNR=ECI']]
x_werte = x_werte.join(assetdata_returns.iloc[:,13], how = 'left')
x_werte = x_werte.join(macrodata_returns[['USRSL=ECI', 'USIP=ECI']], how = 'left')
x_werte = x_werte.join(macrodata_returns['XTIMVA01NLM667S'], how = 'left')
x_werte = x_werte.join(macrodata_returns['EUPPI=ECI'], how = 'left')
x_werte = x_werte.join(macrodata_returns['DEGDP=ECI'], how = 'left')
x_werte = x_werte.join(macrodata_returns['EUGDP=ECI'], how = 'left')
x_werte = x_werte.join(assetdata_returns.iloc[:,16], how = 'left')

In [ ]:
asset = {'Apple Inc': [0,1,1],'NVIDIA Corp': [0,1,1], 'Alphabet Inc': [1,2,1], 'Cisco Systems Inc': [2,5,3], 'ASML Holding NV': [5,6,0], 'ASML Holding NV EU': [6,7,1],'SAP SE': [7,8,0],'SAP SE EU': [8,9,0], 'Logitech International SA': [9,10,1]}
anz_x_werte = pd.DataFrame(data=asset, index = ['Index von', 'Index bis', 'Anzahl'])

In [ ]:
import statsmodels.api as sm

modellenamen_selektiert = ['fit_apple_select','fit_nvidia_select', 'fit_alphabet_select', 'fit_cisco_select', 'fit_asml_select','fit_asml_eu_select','fit_sap_select','fit_sap_eu_select', 'fit_logitech_select']
    
for i in range(0,len(y_werte.columns)):
    y = y_werte.iloc[:,i]
    
    von = anz_x_werte.iloc[0,i]
    bis = anz_x_werte.iloc[1,i]
    
    x = x_werte.iloc[1:,von:bis]
    
    x = sm.add_constant(x)
        
    exec(f'{modellenamen_selektiert[i]} = sm.OLS(y.astype(float),x.astype(float), missing = "drop").fit()')

## 5.7 Variablensignifikant prüfen (selektiertes Modell)

In [ ]:
modellenamen_selektiert = ['fit_apple_select','fit_nvidia_select', 'fit_alphabet_select', 'fit_cisco_select', 'fit_asml_select','fit_asml_eu_select','fit_sap_select','fit_sap_eu_select', 'fit_logitech_select']

for i in range(0,len(modellenamen_selektiert)):
    exec(f'print({modellenamen_selektiert[i]}.summary())')

## 5.8 adj. R^2 vergleichen

In [ ]:
modellenamen_selektiert = ['fit_apple_select','fit_nvidia_select', 'fit_alphabet_select', 'fit_cisco_select', 'fit_asml_select','fit_asml_eu_select','fit_sap_select','fit_sap_eu_select', 'fit_logitech_select']

adjusted_r2 = [
                fit_apple_select.rsquared_adj
                ,fit_nvidia_select.rsquared_adj
                ,fit_alphabet_select.rsquared_adj
                ,fit_cisco_select.rsquared_adj
                ,fit_asml_select.rsquared_adj
                ,fit_sap_select.rsquared_adj
                ,fit_logitech_select.rsquared_adj
                ,fit_asml_eu_select.rsquared_adj
                ,fit_sap_eu_select.rsquared_adj
            ]

modelle_adjusted_r2 = pd.DataFrame(data=adjusted_r2, index = modellenamen_selektiert, columns = ['Adj. R-squared'])

In [ ]:
sns.catplot(data = modelle_adjusted_r2.transpose(), kind = "point", linestyles = "-" ,aspect = 2.8)

## 5.9 Residuen analysieren

In [ ]:
modellenamen_selektiert = ['fit_apple_select','fit_nvidia_select', 'fit_alphabet_select', 'fit_cisco_select', 'fit_asml_select','fit_asml_eu_select','fit_sap_select','fit_sap_eu_select', 'fit_logitech_select']

residuals = fit_apple_select.resid.to_frame().join(fit_nvidia_select.resid.to_frame(),how = 'left', rsuffix = 'nvidia')
residuals = residuals.join(fit_alphabet_select.resid.to_frame(),how = 'left')
residuals = residuals.join(fit_cisco_select.resid.to_frame(),how = 'left', rsuffix = 'cisco')
residuals = residuals.join(fit_asml_select.resid.to_frame(),how = 'left')
residuals = residuals.join(fit_sap_select.resid.to_frame(),how = 'left', rsuffix = 'sap')
residuals = residuals.join(fit_logitech_select.resid.to_frame(),how = 'left')
residuals = residuals.join(fit_asml_eu_select.resid.to_frame(),how = 'left', rsuffix = 'asml_eu')
residuals = residuals.join(fit_sap_eu_select.resid.to_frame(),how = 'left')

residuals.columns = modellenamen_selektiert

### 5.9.1 Normalverteilung

In [ ]:
#Mathematisch
jarque_bera_res(residuals)

In [ ]:
#Optisch (QQ Plot)
import statsmodels.api as smi
import pylab
import matplotlib.pyplot as plt

for i in range(0,len(residuals.columns)):
    x = residuals.iloc[: , i]
    smi.qqplot(x, line = "r")
    plt.title(x.name)
    pylab.show()

In [ ]:
residuals = residuals.drop(['fit_nvidia_select','fit_asml_eu_select' ,'fit_logitech_select'], 1)

### 5.9.2 Heteroskedastizität

In [ ]:
#Mathematisch
from statsmodels.compat import lzip
import statsmodels.stats.api as sms

breusch_apple = sms.het_breuschpagan(fit_apple_select.resid, fit_apple_select.model.exog)[1]
breusch_alphabet = sms.het_breuschpagan(fit_alphabet_select.resid, fit_alphabet_select.model.exog)[1]
breusch_cisco = sms.het_breuschpagan(fit_cisco_select.resid, fit_cisco_select.model.exog)[1]
breusch_asml = sms.het_breuschpagan(fit_asml_select.resid, fit_asml_select.model.exog)[1]
breusch_sap = sms.het_breuschpagan(fit_sap_select.resid, fit_sap_select.model.exog)[1]
breusch_sap_eu = sms.het_breuschpagan(fit_sap_eu_select.resid, fit_sap_eu_select.model.exog)[1]


breusch = pd.DataFrame([breusch_apple,breusch_alphabet,breusch_cisco,breusch_asml,breusch_sap_eu,breusch_sap], index = residuals.columns)

for i in range(0,len(residuals.columns)):
    if breusch.iloc[i,0] <= 0.05:
        print("Der p-Value von " + str(breusch.index[i]) + " ist %f" % breusch.iloc[i,0] + ". Somit herrscht Heteroskedastizität.")
        
print("")
print("Der Rest ist gut.")

In [ ]:
#Optisch
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

for i in range(0,len(residuals.columns)):
    x = residuals.iloc[:,i]
    x = np.reshape(x, len(x))

    plt.plot(x, color = 'black',label = 'Residuals')
    plt.title(residuals.columns[i])
    plt.show()

In [ ]:
residuals = residuals.drop(['fit_sap_select','fit_sap_eu_select'], 1)

### 5.9.3 Autokorrelation

In [ ]:
from statsmodels.stats.stattools import durbin_watson

for i in range(0,len(residuals.columns)):
    durbin = durbin_watson(residuals.iloc[:,i].dropna())
    if durbin > 1.5 and durbin < 2.5:
        print("Der Teststatistik von " + str(residuals.columns[i]) + " ist %f" % durbin + ". Somit herrscht keine Autokorrelation.")

### 5.9.4 MSE

In [ ]:
y_true = assetdata_returns.iloc[:,[0,2,3,4]]

In [ ]:
y_hat_apple = fit_apple_select.predict()
y_hat_apple = pd.DataFrame(y_hat_apple)

y_hat_alphabet = fit_alphabet_select.predict()
y_hat_alphabet = pd.DataFrame(y_hat_alphabet)

y_hat_cisco = fit_cisco_select.predict()
y_hat_cisco = pd.DataFrame(y_hat_cisco)

y_hat_asml = fit_asml_select.predict()
y_hat_asml = pd.DataFrame(y_hat_asml)

y_hat = y_hat_apple.join(y_hat_alphabet, how = 'left', rsuffix = '1').join(y_hat_cisco, how = 'left').join(y_hat_asml, how = 'left', rsuffix = '1')
y_hat.index = y_true.index

y_hat.columns = ['Apple Inc Predict', 'Alphabet Inc Predict', 'Cisco Systems Inc Predict', 'ASML Holding NV Predict']

In [ ]:
#Optiscch
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

for i in range(0,len(y_true.columns)):
    x = y_true.iloc[:,i]
    x = np.reshape(x, len(x))
    y = y_hat.iloc[:,i]
    y = np.reshape(y, len(y))

    plt.plot(x, color = 'black',label = y_true.columns[i])
    plt.plot(y, color = 'red', label = y_hat.columns[i])
    plt.legend(loc = 'best')
    plt.show()

In [ ]:
#Mathematisch
modellenamen_selektiert = ['fit_apple_select', 'fit_alphabet_select', 'fit_cisco_select', 'fit_asml_select']
    
for i in range(0,len(modellenamen_selektiert)):    
    exec(f'mse = {modellenamen_selektiert[i]}.mse_total')
    print('Das ' + str(y_true.columns[i])+ ' Modell hat einen MSE von ' + str(mse) +'.')

## 5.10 Rollingregreession machen (selektiertes Modell)

### 5.10.1 Daten anpassen

In [ ]:
y_werte_neu = y_werte.drop(['NVIDIA Corp','ASML Holding NV EU' ,'SAP SE', 'SAP SE EU', 'Logitech International SA'], 1)

In [ ]:
x_werte_neu = x_werte.drop(['EUPPI=ECI','DEGDP=ECI' ,'EUGDP=ECI', 'Logitech International SA Lag'], 1)

In [ ]:
asset = {'Apple Inc': [0,1,1], 'Alphabet Inc': [1,2,1], 'Cisco Systems Inc': [2,5,3], 'ASML Holding NV': [5,6,0]}
anz_x_werte_neu = pd.DataFrame(data=asset, index = ['Index von', 'Index bis', 'Anzahl'])

### 5.10.2 manuelle Methode

In [ ]:
import statsmodels.api as sm

modellenamen_selektiert_rolling = ['fit_apple_select_rolling', 'fit_alphabet_select_rolling', 'fit_cisco_select_rolling', 'fit_asml_select_rolling']
    
for i in range(0,len(modellenamen_selektiert_rolling)):
    y = y_werte_neu.iloc[:,i]
    
    von = anz_x_werte_neu.iloc[0,i]
    bis = anz_x_werte_neu.iloc[1,i]
    
    x = x_werte_neu.iloc[1:,von:bis]
    
    x = sm.add_constant(x)
    
    exec(f'{modellenamen_selektiert_rolling[i]} = rolling_regression(y.astype(float), x.astype(float), 30)')
    exec(f'{modellenamen_selektiert_rolling[i]} = pd.DataFrame.from_dict(dict(zip({modellenamen_selektiert_rolling[i]}.index, {modellenamen_selektiert_rolling[i]}.values))).T')

In [ ]:
fit_apple_select_rolling.columns=["Beta_1","Beta_2"]
fit_alphabet_select_rolling.columns=["Beta_1","Beta_2"]
fit_cisco_select_rolling.columns=["Beta_1","Beta_2","Beta_3","Beta_4"]
fit_asml_select_rolling.columns = ["Beta_1","Beta_2"]

In [ ]:
import chart_studio
import cufflinks as cf

cf.set_config_file(offline = True)

fit_apple_select_rolling.normalize().iplot()
fit_alphabet_select_rolling.normalize().iplot()
fit_cisco_select_rolling.normalize().iplot()
fit_asml_select_rolling.normalize().iplot()

### 5.10.3 package Methode

In [ ]:
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

modellenamen_selektiert_rolling = ['fit_apple_select_rolling', 'fit_alphabet_select_rolling', 'fit_cisco_select_rolling', 'fit_asml_select_rolling']
    
for i in range(0,len(modellenamen_selektiert_rolling)):
    y = y_werte_neu.iloc[:,i].values
    
    von = anz_x_werte_neu.iloc[0,i]
    bis = anz_x_werte_neu.iloc[1,i]
    
    x = x_werte_neu.iloc[1:,von:bis]
    x = sm.add_constant(x)
        
    rols = RollingOLS(y.astype(float), x.astype(float), window=30)
    rres = rols.fit()
    
    params = rres.params.copy()
    params.index = np.arange(1, params.shape[0] + 1)
    
    exec(f'{modellenamen_selektiert_rolling[i]}_2 = params')
    exec(f'{modellenamen_selektiert_rolling[i]}_plot_2 = rres')

In [ ]:
fig = plt.figure(figsize=(14,6))
fig = fit_apple_select_rolling_plot_2.plot_recursive_coefficient(fig=fig)

In [ ]:
fig = plt.figure(figsize=(14,6))
fig = fit_alphabet_select_rolling_plot_2.plot_recursive_coefficient(fig=fig)

In [ ]:
fig = plt.figure(figsize=(14,6))
fig = fit_cisco_select_rolling_plot_2.plot_recursive_coefficient(fig=fig)

In [ ]:
fig = plt.figure(figsize=(14,6))
fig = fit_asml_select_rolling_plot_2.plot_recursive_coefficient(fig=fig)

### 5.11 Chow Test

In [ ]:
modellenamen_chow = ['Apple Modell', 'Alphabet Modell', 'Cisco Modell', 'ASML Modell']
import scipy

for i in range(0,len(modellenamen_chow)):
    von = anz_x_werte_neu.iloc[0,i]
    bis = anz_x_werte_neu.iloc[1,i]

    y = y_werte_neu.iloc[:,i]    
    x = x_werte_neu.iloc[1:,von:bis]
    x = sm.add_constant(x)
    
    fit_1 = sm.OLS(y.astype(float),x.astype(float), missing = "drop").fit()
    rssd = fit_1.ssr
    
    y_1 = y_werte_neu.iloc[:60,i]
    x_1 = x_werte_neu.iloc[1:61,von:bis]
    x_1 = sm.add_constant(x_1)    
    
    fit_2 = sm.OLS(y_1.astype(float),x_1.astype(float), missing = "drop").fit()
    rssb = fit_2.ssr
    
    y_2 = y_werte_neu.iloc[60:,i]
    x_2 = x_werte_neu.iloc[61:,von:bis]
    x_2 = sm.add_constant(x_2)
    
    fit_3 = sm.OLS(y_2.astype(float),x_2.astype(float), missing = "drop").fit()
    rssnb = fit_3.ssr
    
    j = x.shape[1]
    k = x_1.shape[1]
    n1 = x_1.shape[0]
    n2 = x_2.shape[0]
    
    werte = (j,k,n1,n2)
    
    chow_statistic = ((rssd-(rssb+rssnb))/j)/((rssb+rssnb)/(n1+n2-2*k))
    chow_wert = scipy.stats.f.cdf(chow_statistic, j, (n1+n2-2*k))

    print('Die Werte für das ' + modellenamen_chow[i] + ' sind ' + str(werte) + ' und der Chow Test zeigt einen Wert von ' + str(chow_wert) + ' an.')

# 6 Daten speichern

In [ ]:
#In Excel
import openpyxl
assetdata.to_excel("assetdata.xlsx")
assetdata_returns.to_excel("assetdata_returns.xlsx")

macrodata.to_excel("macrodata.xlsx")
macrodata_returns.to_excel("macrodata_returns.xlsx")

macrodata_us.to_excel("macrodata_us.xlsx")
macrodata_nl.to_excel("macrodata_nl.xlsx")
macrodata_eu.to_excel("macrodata_eu.xlsx")
macrodata_ch.to_excel("macrodata_ch.xlsx")
macrodata_de.to_excel("macrodata_de.xlsx")
macrodata_cn.to_excel("macrodata_cn.xlsx")
macrodata_jp.to_excel("macrodata_jp.xlsx")
macrodata_sk.to_excel("macrodata_sk.xlsx")

residuals.to_excel("residuals.xlsx")

total.to_excel("total.xlsx")

In [ ]:
modellenamen = ['fit_apple', 'fit_nvidia', 'fit_alphabet', 'fit_cisco', 'fit_asml', 'fit_sap', 'fit_logitech', 'fit_tencent', 'fit_samsung', 'fit_sony', 'fit_asml_eu', 'fit_sap_eu', 'fit_logitech_eu']

In [ ]:
#In Text
modellenamen = ['fit_apple', 'fit_nvidia', 'fit_alphabet', 'fit_cisco', 'fit_asml', 'fit_sap', 'fit_logitech', 'fit_tencent', 'fit_samsung', 'fit_sony', 'fit_asml_eu', 'fit_sap_eu', 'fit_logitech_eu']

with open('summary_erste_modelle.txt', 'w') as output:
    for i in range(0,len(modellenamen)):
        exec(f'output.write({modellenamen[i]}.summary().as_text())')

In [ ]:
#In Text
modellenamen_selektiert = ['fit_apple_select','fit_nvidia_select', 'fit_alphabet_select', 'fit_cisco_select', 'fit_asml_select','fit_asml_eu_select','fit_sap_select','fit_sap_eu_select', 'fit_logitech_select']

with open('summary_selekt_modelle.txt', 'w') as output:
    for i in range(0,len(modellenamen_selektiert)):
        exec(f'output.write({modellenamen_selektiert[i]}.summary().as_text())')

In [ ]:
#In SQL
import pandas as pd
import sqlite3
conn = sqlite3.connect('AQM.db')

c = conn.cursor()
c.execute('DROP TABLE IF EXISTS assetdata')
assetdata.to_sql('assetdata', con=conn, index = False)

c.execute('DROP TABLE IF EXISTS assetdata_returns')
assetdata_returns.to_sql('assetdata_returns', con=conn, index = False)

c.execute('DROP TABLE IF EXISTS macrodata')
macrodata.to_sql('macrodata', con=conn, index = False)

c.execute('DROP TABLE IF EXISTS macrodata_returns')
macrodata_returns.to_sql('macrodata_returns', con=conn, index = False)

# 7 Sonst

In [ ]:
acf_plot(assetdata,12)